# Getting Started: Building Agents over LlamaCloud RAG Pipelines

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/getting_started_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to build a function calling agent (powered by Claude) over RAG pipelines built with LlamaCloud.

Adding an agentic layer to RAG allows you to build in a layer of query planning and state management that allows you to ask multi-part complex questions over existing RAG pipelines and get back answers in a conversational manner.


## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the Anthropic API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install llama-index-llms-anthropic
%pip install llama-index-indices-llama-cloud
%pip install llama-index

In [1]:
import nest_asyncio
nest_asyncio.apply()

Make sure your ANTHROPIC_API_KEY is set. Otherwise explicitly specify the `api_key` parameter.

In [2]:
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model="claude-3-opus-20240229")

## Build Two LlamaCloud Indexes

Our data sources are the 2021 Lyft and Uber 10K documents.

In contrast to the other getting started examples, in this notebook we will build **two** RAG pipelines: one for Uber and one for Lyft. This is for the sake of example; we can plug in both RAG pipelines as tools for the agent to reason over.

To create each index, follow the instructions:
1. You can download them here ([Uber 10K](https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1), [Lyft 10K](https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1))
2. Follow instructions on `https://cloud.llamaindex.ai/` to signup for an account. Create a pipeline by uploading these documents.

In [3]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index_uber = LlamaCloudIndex(
  name="<index_uber>", 
  project_name="<project_name>",
  api_key="llx-"
)
index_lyft = LlamaCloudIndex(
  name="<index_lyft>", 
  project_name="<project_name>",
  api_key="llx-"
)

For each index, get a query engine from the index, which gives us an out-of-the-box RAG pipeline.

In [4]:
query_engine_uber = index_uber.as_query_engine(llm=llm)
query_engine_lyft = index_lyft.as_query_engine(llm=llm)

## Anthropic Agent over LlamaCloud RAG Pipelines

We convert both query engines to tools and pass it to a function calling Anthropic Opus agent.

In [5]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tool_uber = QueryEngineTool(
    query_engine=query_engine_uber,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)
tool_lyft = QueryEngineTool(
    query_engine=query_engine_lyft,
    metadata=ToolMetadata(
        name="lyft_10k",
        description=(
            "Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [8]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [tool_uber, tool_lyft], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
response = agent.chat("Tell me both the tailwinds for Uber and Lyft?")
print(str(response))

Added user message to memory: Tell me both the tailwinds for Uber and Lyft?
=== LLM Response ===
<thinking>
To answer this question about tailwinds for both Uber and Lyft, I will need to query information from both the uber_10k and lyft_10k tools. The input parameter should be a question asking about tailwinds or positive factors mentioned in the 10-K filings.
</thinking>
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What were some of the tailwinds or positive factors mentioned for Uber's business in 2021?"}
=== Function Output ===
Based on the context provided, a few positive factors for Uber's business in 2021 were:

1. Uber experienced less seasonality in 2021 as a result of the COVID-19 pandemic and related restrictions, which accelerated the growth of Uber's Delivery business as cities imposed various dining restrictions. 

2. Uber's Delivery offering was attracting new consumers to their platform. Over 60% of first-time Delivery consumers in Q4 2021 we